In [1]:
import io
import pandas as pd
import requests

In [2]:
url = 'https://storage.googleapis.com/uber-data-engineering-project/uber_data.csv'
response = requests.get(url)

In [3]:
df = pd.read_csv(io.StringIO(response.text), sep=',')

In [4]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

KeyError: 'tpep_pickup_datetime'

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

In [ ]:
df.head()

In [ ]:
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday


datetime_dim['datetime_id'] = datetime_dim.index

# datetime_dim = datetime_dim.rename(columns={'tpep_pickup_datetime': 'datetime_id'}).reset_index(drop=True)
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]
#
datetime_dim.head()

In [ ]:
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]

trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]


In [ ]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]

# rate_code_dim.head()

In [ ]:
rate_code_dim.head()

In [ ]:
pickup_location_dim = df[['pickup_longitude', 'pickup_latitude']].reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id','pickup_latitude','pickup_longitude']] 


dropoff_location_dim = df[['dropoff_longitude', 'dropoff_latitude']].reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','dropoff_latitude','dropoff_longitude']]

In [ ]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]

In [ ]:

fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

In [ ]:
payment_type_dim.columns

In [ ]:
fact_table.columns

In [ ]:
fact_table

In [ ]:
# CREATE OR REPLACE TABLE `data-with-themihirmathur.uber_data_engineering_by_mihir.tbl_analysis` AS (
# SELECT
#   f.VendorID,
#   f.tpep_pickup_datetime,
#   f.tpep_dropoff_datetime,
#   p.passenger_count,
#   td.trip_distance,
#   rc.RatecodeID,
#   f.store_and_fwd_flag,
#   pl.pickup_latitude,
#   pl.pickup_longitude,
#   dl.dropoff_latitude,
#   dl.dropoff_longitude,
#   pt.payment_type,
#   f.fare_amount,
#   f.extra,
#   f.mta_tax,
#   f.tip_amount,
#   f.tolls_amount,
#   f.improvement_surcharge,
#   f.total_amount
# FROM
#   `data-with-themihirmathur.uber_data_engineering_by_mihir.fact_table` f
#   JOIN `data-with-themihirmathur.uber_data_engineering_by_mihir.passenger_count_dim` p ON f.passenger_count_id = p.passenger_count_id
#   JOIN `data-with-themihirmathur.uber_data_engineering_by_mihir.trip_distance_dim` td ON f.trip_distance_id = td.trip_distance_id
#   JOIN `data-with-themihirmathur.uber_data_engineering_by_mihir.rate_code_dim` rc ON f.rate_code_id = rc.rate_code_id
#   JOIN `data-with-themihirmathur.uber_data_engineering_by_mihir.pickup_location_dim` pl ON f.pickup_location_id = pl.pickup_location_id
#   JOIN `data-with-themihirmathur.uber_data_engineering_by_mihir.dropoff_location_dim` dl ON f.dropoff_location_id = dl.dropoff_location_id
#   JOIN `data-with-themihirmathur.uber_data_engineering_by_mihir.payment_type_dim` pt ON f.payment_type_id = pt.payment_type_id);